# Falcon-7b Sentiment Analysis

# Installation

In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [16]:
notebook_login()

In [17]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/CZ4042 NN Grp Proj/falcon-7b")
#os.chdir("/content/drive/My Drive/Y4S1/CZ4042 NN Grp Proj/falcon-7b")
print("Changed to:", os.getcwd())  # Print the new current directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed to: /content/drive/.shortcut-targets-by-id/15Egk8g4ohv4wHfYv2tOw0YY-UUnwHGzF/CZ4042 NN Grp Proj/falcon-7b


# Data prep

In [18]:
from datasets import load_dataset

# Load the dataset
data = load_dataset("csv", data_files="IMDB Dataset.csv")
data_train = data['train'].select([i for i in range(400)])

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd

filename = "IMDB Dataset.csv"

df = pd.read_csv(filename)
df = df.head(500)
train_df, test_df = train_test_split(df, test_size=0.2)

#reset index
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

#X_test and y_test df
X_test = test_df["review"]
y_test = test_df["sentiment"]

# Load Falcon Model & Tokenizer

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)nstruct-sharded/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)ded/resolve/main/configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)-sharded/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)arded/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)ruct-sharded/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)ded/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model (1 prompt)

In [ ]:
model_original = model

In [ ]:
prompt = """
<human>: Analyze the sentiment of the movie review below,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or "negative".
            Review: I thought this is movie is good, but it was actually trash!!
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<human>: Analyze the sentiment of the movie review below,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or "negative".
            Review: I thought this is movie is good, but it was actually trash!!
<assistant>: Negative
User 
CPU times: user 2.71 s, sys: 312 ms, total: 3.03 s
Wall time: 5.7 s


# Prompt generation

In [22]:
import numpy as np
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the movie review enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label 'positive' or 'negative' only.

            Review: [{data_point["review"]}]
            Sentiment = {data_point["sentiment"]}

            """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

def generate_test_prompt(review):
    return f"""
            Analyze the sentiment of the movie review enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label 'positive' or 'negative' only.

            Review: [{review}]
            Sentiment =
             """.strip()

In [23]:
data_train = data_train.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

# Test original model

In [21]:
from tqdm import tqdm
from transformers import pipeline

def predict(X_test, model, tokenizer, generation_config):
  y_pred = []
  device = "cuda:0"

  for i in tqdm(range(len(X_test))):
      prompt = generate_test_prompt(X_test[i])

      encoding = tokenizer(prompt, return_tensors="pt").to(device)
      with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
      if outputs:
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
      if result:
        answer = result.split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
      else:
          y_pred.append("none")
  return y_pred

def evaluate(y_true, y_pred):
    labels = ['positive', 'negative']
    mapping = {'positive': 1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 3
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

y_pred_original = predict(X_test, model, tokenizer, generation_config)

100%|██████████| 100/100 [06:03<00:00,  3.63s/it]


In [ ]:
print("Results without fine-tuning")
evaluate(y_test, y_pred_original)

Results without fine-tuning
Accuracy: 0.660
Accuracy for label 0: 0.358
Accuracy for label 1: 1.000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.36      0.53        53
           1       0.58      1.00      0.73        47

    accuracy                           0.66       100
   macro avg       0.79      0.68      0.63       100
weighted avg       0.80      0.66      0.62       100


Confusion Matrix:
[[19 34  0]
 [ 0 47  0]
 [ 0  0  0]]


# Finetune the model

In [24]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data_train,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.805900
2,3.006300
3,2.919500
4,2.877500
5,2.990200
6,2.824300
7,2.941200
8,2.968500
9,2.971400
10,2.881900


TrainOutput(global_step=100, training_loss=2.37931565284729, metrics={'train_runtime': 991.5645, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.101, 'total_flos': 2933558268546048.0, 'train_loss': 2.37931565284729, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("falcon-7b-trained-model-5")

In [ ]:
PEFT_MODEL = "fabecode/falcon-7b-movie-sentiments-5" #replace with your own fine-tuned model

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

In [ ]:
# Use below to load finetuned model directly
# PEFT_MODEL = "fabecode/falcon-7b-movie-sentiments-4"
# config = PeftConfig.from_pretrained(PEFT_MODEL)
# model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     return_dict=True,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True
# )

# tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# tokenizer.pad_token = tokenizer.eos_token

# model = PeftModel.from_pretrained(model, PEFT_MODEL)

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 3
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

y_pred_finetuned = predict(X_test, model, tokenizer, generation_config)

100%|██████████| 100/100 [06:03<00:00,  3.63s/it]


In [25]:
print("Results with fine-tuning")
evaluate(y_test, y_pred_finetuned)

Results with fine-tuning
Accuracy: 0.820
Accuracy for label 0: 0.660
Accuracy for label 1: 1.000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.66      0.80        53
           1       0.72      1.00      0.84        47

    accuracy                           0.82       100
   macro avg       0.86      0.83      0.82       100
weighted avg       0.87      0.82      0.82       100


Confusion Matrix:
[[35 18  0]
 [ 0 47  0]
 [ 0  0  0]]
